In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

import pyspark.pipelines as dp

# DLT Pipeline


streaming table and setting up expectations for our data


In [0]:
@dp.table(name= "dim_products_stage")

@dp.expect_all({
    "non_null_id":"product_id is NOT NULL",
    "non_null_name":"product_name is NOT NULL"
})
def dim_products_stage():

    df = spark.readStream.table("aws_db_retail_metastore.silver.products_silver")
    return df

In [0]:
@dp.view(name = "dim_products_view")

def dim_products_view():
    df = spark.readStream.table("Live.dim_product_stage")
    return df


defining SCD type 2 on products table using DLT on dim_products_stage 

In [0]:
#creating an empty streaming table

dp.create_streaming_table("dim_products")

In [0]:
dp.create_auto_cdc_flow(
  target = "dim_products",
  source = "Live.dim_products_view",
  keys = ["product_id"],
  sequence_by = "product_id",
  stored_as_scd_type = 2,
)